# Scraping Top Repositories for Topics on GitHub

- Introduction about web scraping
- Introduction about GitHub and the problem statement
- Tools used - (Python, requests, Beautiful Soup, Pandas)

PROJECT OUTLINE:

- We're going to scrape https://github.com/topics
- We'll get a list of topics. For each topic, we'll get topic title, topic page URL and topic description
- For each topic, we'll get the top repositories in the topic from the first topic page
- For each repository, we'll grab the repo name, username, stars and repo URL
- For each topic we'll create a CSV file in the following format:

```
Repo Name,Username,Stars,Repo URL
three.js,mrdoob,69700,https://github.com/mrdoob/three.js
libgdx,libgdx,18300,https://github.com/libgdx/libgdx
```

## Scrape the list of topics from Github


- Use requests to downlaod the page
- Use BS4 to parse and extract information
- convert to a Pandas dataframe

Let's write a function to download the page.

In [102]:
import requests
from bs4 import BeautifulSoup

def get_topics_page():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    return doc

Get_topics_page takes the url of the webpage and returns the doc file is bs4 format

In [103]:
doc = get_topics_page()
type(doc)

bs4.BeautifulSoup

Creating some helper functions to parse information from the page.

To get topic titles, we can pick `p` tags with the `class` ...

![](https://i.imgur.com/OnzIdyP.png)


In [104]:
def get_topic_titles(doc):
    title_selector = 'f3 lh-condensed mb-0 mt-1 Link--primary'
    topic_title_tags = doc.find_all('p', {'class' : title_selector})
    
    topic_titles =[]
    for tag in topic_title_tags:
        topic_titles.append(tag.text)
    return topic_titles


`get_topic_titles` can be used to get the list of titles

In [105]:
titles = get_topic_titles(doc)

In [106]:
len(titles)

30

In [107]:
titles[:5]

['3D', 'Ajax', 'Algorithm', 'Amp', 'Android']

Similarly we have defined functions for descriptions and URLs.

In [108]:
def get_topic_descs(doc):
    desc_selector = 'f5 color-fg-muted mb-0 mt-1'
    topic_desc_tags = doc.find_all('p', {'class': desc_selector})
    topic_descs = []
    for tag in topic_desc_tags:
        topic_descs.append(tag.text.strip())
    return topic_descs

get_topic_desc takes the bs4 doc and returns a list of topic descriptions

In [109]:
def get_topic_urls(doc):
    topic_link_tags = doc.find_all('a', {'class': 'no-underline flex-1 d-flex flex-column'})
    topic_urls = []
    base_url = 'https://github.com'
    for tag in topic_link_tags:
        topic_urls.append(base_url + tag['href'])
    return topic_urls

get_topic_urls takes the bs4 doc and returns a list of topic urls

Let's put this all together into a single function

In [110]:
import pandas as pd

In [111]:
def scrape_topics():
    topics_url = 'https://github.com/topics'
    response = requests.get(topics_url)
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(topic_url))
    doc = BeautifulSoup(response.text, 'html.parser')
    topics_dict = {
        'title': get_topic_titles(doc),
        'description': get_topic_descs(doc),
        'url': get_topic_urls(doc)
    }
    return pd.DataFrame(topics_dict)

## Get the top 25 repositories from a topic page


In [112]:
def get_topic_page(topic_url):
    
    #Download the page
    response = requests.get(topic_url)
    
    #Check successful response
    if response.status_code != 200:
        raise Exception('Failed to load page {}' .format(topic_url))
    
    # Parse using beautiful soup
    topic_doc = BeautifulSoup(response.text, 'html.parser')
    return topic_doc


In [113]:
doc = get_topic_page('https://github.com/topics/3d')

get_repo_info returns all the information about a repository

In [114]:
def parse_star_count(stars):
    stars=stars.strip()
    if stars[-1]=='k':
        return int(float(stars[:-1])*1000)
    return(int(stars))

In [115]:
def get_repo_info(h1_tag, star_tag):
    base_url = "https://github.com"
    a_tags = h1_tag.find_all('a')
    username = a_tags[0].text.strip()
    repo_name = a_tags[1].text.strip()
    repo_url = base_url + a_tags[1]['href']
    stars = parse_star_count(star_tag.text.strip())
    return username, repo_name, stars, repo_url

In [116]:
def get_topic_repos(topic_doc):
    # Get the h3 tags containg repo name, repo url and username
    h3_selection_class = 'f3 color-fg-muted text-normal lh-condensed'
    repo_tags = topic_doc.find_all('h3',{'class': h3_selection_class})
    
    #Get the star tags
    star_selection_class = 'Counter js-social-count'
    star_tags = topic_doc.find_all('span',{'class':star_selection_class})
    
    topic_repos_dict ={'username':[], 
                       'repo_name':[], 
                       'stars':[], 
                       'repo_url':[]}
    
    # Get repo info
    for i in range(len(repo_tags)):
        repo_info = get_repo_info(repo_tags[i],star_tags[i])
        topic_repos_dict['username'].append(repo_info[0])
        topic_repos_dict['repo_name'].append(repo_info[1])
        topic_repos_dict['stars'].append(repo_info[2])
        topic_repos_dict['repo_url'].append(repo_info[3])
    
    return pd.DataFrame(topic_repos_dict)

get_topic_repos function returns a dataframe consisting of the username, 
repo name, the number of stars and the repo url of a repository

In [117]:
def scrape_topic(topic_url, topic_name):
    topic_df = get_topic_repos(get_topic_page(topic_url))
    topic_df.to_csv(topic_name + '.csv', index=None)

## Putting it all together

- We have a funciton to get the list of topics
- We have a function to create a CSV file for scraped repos from a topics page
- Let's create a function to put them together

In [118]:
def scrape_topics_repos():
    print('Scraping list of topics')
    topics_df = scrape_topics()
    for index, row in topics_df.iterrows():
        print('Scraping top repositories for "{}"'.format(row['title']))
        scrape_topic(row['url'],row['title'])

Let's run it to scrape the top repos for the all the topics on the first page of https://github.com/topics

In [119]:
scrape_topics_repos()

Scraping list of topics
Scraping top repositories for "3D"
Scraping top repositories for "Ajax"
Scraping top repositories for "Algorithm"
Scraping top repositories for "Amp"
Scraping top repositories for "Android"
Scraping top repositories for "Angular"
Scraping top repositories for "Ansible"
Scraping top repositories for "API"
Scraping top repositories for "Arduino"
Scraping top repositories for "ASP.NET"
Scraping top repositories for "Atom"
Scraping top repositories for "Awesome Lists"
Scraping top repositories for "Amazon Web Services"
Scraping top repositories for "Azure"
Scraping top repositories for "Babel"
Scraping top repositories for "Bash"
Scraping top repositories for "Bitcoin"
Scraping top repositories for "Bootstrap"
Scraping top repositories for "Bot"
Scraping top repositories for "C"
Scraping top repositories for "Chrome"
Scraping top repositories for "Chrome extension"
Scraping top repositories for "Command line interface"
Scraping top repositories for "Clojure"
Scrapin

We can check that the CSVs were created properly


In [120]:
import jovian

In [121]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "jyotiradityabarsain07/scraping-github-topics-repositories" on https://jovian.com
[jovian] Committed successfully! https://jovian.com/jyotiradityabarsain07/scraping-github-topics-repositories


'https://jovian.com/jyotiradityabarsain07/scraping-github-topics-repositories'